In [1]:
import cv2
import mediapipe as mp
import numpy as np

import os
import pickle

# Preprocessing des datas 

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def get_landmarks(image_rgb, pose_model): 
    '''
    Récupère les landmarks d'une image donnée en RGB via un modèle déjà chargé.
    Return: un numpy array (33,4) ou None si rien n'est trouvé.
    '''

    results = pose_model.process(image_rgb)
    
    if results.pose_landmarks:

        pose_np = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark])
        return pose_np

    return None

In [3]:
def get_dataset(folder_path, path_save, pickle_name="dataset.pkl"): 
    ''' récupère les landmarks d'images dans un dossier via la fonction get_landmarks(image)

    folder_path : dossier où sont les images
    path_save   : dossier où sauvegarder le pickle
    pickle_name : nom du fichier pickle

    return: numpy array de shape (N, 132) si flatten (33*4)
    '''

    mp_pose = mp.solutions.pose
    os.makedirs(path_save, exist_ok=True)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        data_list = []
        fichiers = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        fichiers.sort()

        path_save_pickle = os.path.join(path_save, pickle_name)

        for name in fichiers:
            img_path = os.path.join(folder_path, name)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Impossible de lire : {img_path}")
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            landmarks_vector = get_landmarks(img_rgb, pose)

            if landmarks_vector is not None:
                data_list.append(landmarks_vector.flatten())  # (132,)
            else:
                print(f"Ignorée (pas de pose détectée) : {name}")

        dataset = np.array(data_list, dtype=np.float32)

        with open(path_save_pickle, 'wb') as f:
            pickle.dump(dataset, f)

    return dataset

## attributs target  

In [4]:
def get_target(dataset, class_int): 
    ''''récupère la target du dataset'''

    nb_images = dataset.shape[0]
    class_id = class_int
    target = np.full(nb_images, class_id)

    return target 


# Build dataset et concatenisation en numpy array#

In [5]:
def build_full_dataset(data_root, path_save, poses=("cobra", "tree", "downdog","forwardfold","chair",
                                                    "warrior1","warrior2","warrior3","plank","child","lotus")):
    """
    Construit X, y à partir de dossiers:
      data_root/cobra, data_root/tree, data_root/downdog

    Retour:
      X (N,132), y (N,)
    + sauvegarde un pickle par pose (comme vous faites déjà)
    """
    X_list = []
    y_list = []

    for class_id, pose_name in enumerate(poses):
        folder_path = os.path.join(data_root, pose_name)

        ds = get_dataset(
            folder_path=folder_path,
            path_save=path_save,
            pickle_name=f"dataset_{pose_name}.pkl"
        )

        target = get_target(ds, class_id)

        X_list.append(ds)
        y_list.append(target)

    X = np.vstack(X_list).astype(np.float32)
    y = np.concatenate(y_list).astype(np.int64)

    # (optionnel) sauvegarde globale
    with open(os.path.join(path_save, "X_all.pkl"), "wb") as f:
        pickle.dump(X, f)
    with open(os.path.join(path_save, "y_all.pkl"), "wb") as f:
        pickle.dump(y, f)

    return X, y

# Dataset des postures 

In [7]:
path_data = r'C:\Users\mvana\Documents\Formation data scientist\20. ACV\Posture_yoga\data'
path_save = "datasets"

X, y = build_full_dataset(path_data, path_save)
print(X.shape, y.shape)

Ignorée (pas de pose détectée) : 1_364.jpg
Ignorée (pas de pose détectée) : 3_287.jpg
Ignorée (pas de pose détectée) : File26.png
Ignorée (pas de pose détectée) : File57.png
Ignorée (pas de pose détectée) : File59.png
Ignorée (pas de pose détectée) : File63.png
Ignorée (pas de pose détectée) : File68.jpeg
Ignorée (pas de pose détectée) : PXL_20251217_103746696.MP.jpg
Ignorée (pas de pose détectée) : image13.jpeg
Ignorée (pas de pose détectée) : images51.jpg
Ignorée (pas de pose détectée) : 00000104.jpg
Ignorée (pas de pose détectée) : 00000202.jpg
Ignorée (pas de pose détectée) : images133.jpg
Ignorée (pas de pose détectée) : images216.jpg
Ignorée (pas de pose détectée) : images54.jpg
Ignorée (pas de pose détectée) : downdog (16).jpg
Ignorée (pas de pose détectée) : downdog (25).jpg
Ignorée (pas de pose détectée) : downdog (55).jpg
Ignorée (pas de pose détectée) : downdog (68).jpg
Ignorée (pas de pose détectée) : downdog (85).jpg
Ignorée (pas de pose détectée) : downdog (92).jpg
Ignoré

# Vérification de la sortie #

In [8]:
chemin_du_fichier = r'C:\Users\mvana\Documents\Formation data scientist\20. ACV\ACV_yoga\datasets\Y_all.pkl'

with open(chemin_du_fichier, 'rb') as f:
    dataset = pickle.load(f)
dataset.shape

(992,)

In [9]:
get_target(dataset, 1).shape

(992,)